In [1]:
import pandas as pd

df = pd.read_csv("data/stocks_raw_454_posts.csv")

# check for missing values
print("Missing values per column:\\n", df.isnull().sum())
# drop rows where 'Text' is empty (optional)
df = df[df["Text"].str.strip() != ""]
# check for duplicates (based on title + text)
df.drop_duplicates(subset=["Title", "Text"], inplace=True)

# save cleaned data
df.to_csv("data/stocks_cleaned.csv", index=False)
print(f"✅ Cleaned dataset saved: {len(df)} posts")

Missing values per column:\n Title        0
Text         3
Upvotes      0
Comments     0
Timestamp    0
URL          0
dtype: int64
✅ Cleaned dataset saved: 341 posts


In [2]:
# import glob

# csv_files = ["data/stocks_raw_500_posts.csv", "data/stocks_raw_454_posts.csv"]
# df_list = [pd.read_csv(file) for file in csv_files]
# df = pd.concat(df_list, ignore_index=True)

# df.drop_duplicates(subset=["Title", "Text"], inplace=True)

# df.to_csv("data/stocks_combined.csv", index=False)
# print(f"✅ Combined dataset saved: {len(df)} posts")

In [3]:
import certifi
print(certifi.where())

/Users/ullaangell/Desktop/topic_modeling_project/.venv/lib/python3.12/site-packages/certifi/cacert.pem


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ullaangell/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ullaangell/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True